In [94]:
import pandas as pd
from pathlib import Path
import os
import sys
import requests
from PIL import Image
import io
import os
from tqdm import tqdm

import io
from PIL import Image
from IPython.display import display

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [109]:
LLAVA_DIRECTORY_PATH = '/home/user/Documents/GitHub/LLaVA'

sys.path.insert(0,LLAVA_DIRECTORY_PATH)
sys.path.insert(0, Path(LLAVA_DIRECTORY_PATH).joinpath('playground/finetuning').absolute().as_posix())  # For importing the finetuning specific files

from llava.eval.maya.eval_utils import load_maya_model, get_single_sample_prediction, generate_cvqa_prompt

In [9]:
df = pd.read_parquet(cvqa_folder / filelist[0])

In [10]:
df.head()

,image,ID,Subset,Question,Translated Question,Options,Translated Options,Label,Category,Image Type,Image Source,License
0,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,5865910664272406812_1,"('Chinese', 'China')",图中的‘318’代表着什么意思？,What does ‘318’ in the picture mean?,"[海拔, 日期, 路程长度, 国道]","[Altitude, Date, Length of the road, National ...",-1,Vehicles and Transportation,Self,Self-research_only,CC BY-NC
1,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,5865925454277921835_2,"('Filipino', 'Philippines')",Ano ang kilalang tawag sa babae sa larawan?,The woman in the photo is most commonly known as?,"[Ina ng Pilipinas, Ina ng Manila, Ina ng Rebol...","[Mother of the Philippines, Mother of Manila, ...",-1,Traditions / art / history,External,https://commons.wikimedia.org/wiki/File:Melcho...,Public domain
2,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,5865743574274453408_0,"('Korean', 'South Korea')",이 사진에서 사람들이 참여하고 있는 활동은 무엇입니까?,What activity are the people participating in ...,"[마라톤 , 조정 , 댄스 스포츠, 전통 무술]","[Marathon, Rowing, Dance sports, Traditional m...",-1,Sports and recreation,External,https://commons.wikimedia.org/wiki/File:KOCIS_...,CC BY-SA 2.0
3,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,5865940594277100134_1,"('Spanish', 'Spain')",¿Quién pintó este cuadro?,Who painted this picture?,"[Pablo Picasso, Franciso de Goya, Bartolomé Es...","[Pablo Picasso, Franciso de Goya, Bartolome Es...",-1,Traditions / art / history,External,https://commons.wikimedia.org/wiki/File:Maja_d...,Public domain
4,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,5865910684273901143_0,"('Chinese', 'China')",这是什么植物？,What plant is this?,"[郁金香, 樱花, 牡丹花, 玫瑰花]","[Tulip, Cherry blossoms, Peony, Roses]",-1,Plants and animal,External,https://www.flickr.com/photos/a3u9c2/187996110...,Attribution-NoDerivs License


### Loading Model

In [59]:
model_base = 'CohereForAI/aya-23-8B'
model_path = '/home/user/Documents/GitHub/maya_full_ft'
mode = 'finetuned'

maya_model = load_maya_model(model_base, model_path, mode = mode)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of the model checkpoint at /home/user/Documents/GitHub/maya_full_ft were not used when initializing LlavaCohereForCausalLM: ['model.vision_tower.vision_tower.vision_model.embeddings.patch_embedding.bias', 'model.vision_tower.vision_tower.vision_model.embeddings.patch_embedding.weight', 'model.vision_tower.vision_tower.vision_model.embeddings.position_embedding.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.layer_norm1.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.layer_norm1.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.layer_norm2.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.layer_norm2.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.mlp.fc1.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.mlp.fc1.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.mlp.fc2.bias', 'model.vision_tower.vision_tower.vision_model.e

### Testing on Single Sample

In [90]:
index = 0

sample = df.iloc[index]

image = sample.image['bytes']

user_question = generate_cvqa_prompt(sample)

In [91]:
output = get_single_sample_prediction(maya_model, image, user_question)

 3


### Running CVQA eval

In [119]:
cvqa_folder = Path('/home/user/Documents/Datasets/CVQA/')

filelist = os.listdir(cvqa_folder)
if  '.ipynb_checkpoints' in filelist:
    filelist.remove('.ipynb_checkpoints')

In [111]:
results = []
skipped_answers = []

for file in filelist:

    df = pd.read_parquet(cvqa_folder / file)

    for i in tqdm(range(len(df))):

        sample = df.iloc[i]
        
        sample_id = sample.ID
        image = sample.image['bytes']
        user_question = generate_cvqa_prompt(sample)
    
        output = get_single_sample_prediction(maya_model, image, user_question)
    
        try:
            output = int(output)
        except:
            skipped_answers.append([sample_id, output, user_question])
            output = 0
            
        results.append([sample_id, output])




    

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1037/1037 [03:07<00:00,  5.52it/s]


In [113]:
len(results)

10374

In [116]:
resultsDf = pd.DataFrame(results)
resultsDf.columns = ['ID','Prediction'] 
resultsDf.head()

,ID,Prediction
0,5865910664272406812_1,3
1,5865925454277921835_2,0
2,5865743574274453408_0,3
3,5865940594277100134_1,1
4,5865910684273901143_0,0


In [118]:
resultsDf.to_csv('cvqa_predictions.csv',index = False, index_label = False)

In [117]:
os.getcwd()

'/home/user/Documents/GitHub/LLaVA/playground/eval'

#### To cover missed samples

In [ ]:
# existing_result_ids = [x[0] for x in results]

# # results_add = []

# for file in filelist:

#     df = pd.read_parquet(cvqa_folder / file)

#     for i in tqdm(range(len(df))):

#         sample = df.iloc[i]
        
#         sample_id = sample.ID

#         if sample_id in existing_result_ids:
#             continue
        
#         image = sample.image['bytes']
#         user_question = generate_cvqa_prompt(sample)
    
#         output = get_single_sample_prediction(maya_model, image, user_question)
    
#         try:
#             output = int(output)
#         except:
#             skipped_answers.append([sample_id, output, user_question])
#             output = 0
            
#         results.append([sample_id, output])
